<a href="https://colab.research.google.com/github/Wildemberg-Projects/Plants-Diseases/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação de Dependências
# Para utilização do Colab, modifique o '%' por '!'
# As abaixo comentadas são somente para instalar em ambiente local
# %pip install -q -U tensorflow
# %pip install -q -U keras
# %pip install -q -U numpy
# %pip install -q -U pandas
# %pip install -q -U Jinja2
# Uso da GPU precisa ser habilitado pelo comando a seguir
# %pip install tensorflow[and-cuda]
%pip install -q -U tensorflow-addons
%pip install -q -U tensorflow-datasets
%pip install -q -U keras-utils
%pip install -q -U matplotlib


In [2]:
import tensorflow_datasets as tfds
import pandas as pd
from numpy import mean
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import seaborn as sns



In [3]:
# Coletando dados
tf.keras.backend.clear_session()
(ds_train), info = tfds.load('plant_village', split='train', with_info=True)
N_classes_total = 38

# a estrutura de uma imagem (256,256,3), representa a largura e altura em pixels,
# e o 3 representa a quantidade de canais de cores pq é RGB

In [ ]:
# Visualização da Distribuição das Classes
plt.figure(figsize=(10, 6))
sns.countplot(x='label', data=df_train)
plt.title('Distribuição das Labels')
plt.xlabel('Classe')
plt.ylabel('Número de Exemplos')
plt.show()


In [5]:
# Função que será responsável por aplicar um tratamento dos dados para conversão de tipo
def processamento(val):
  image = val['image']
  label = val['label']
  # normalização das imagens convertendo para um tipo que melhor é processado e para um intervalo entre [0, 1]
  image = tf.cast(image, tf.float32) / 255.0
  return image, label


In [9]:
# Filtrar dataframe para somente tomate
N_classes_tomate = 11
def filtrar_tomates(exemplo):
    return exemplo["label"] > 27  # Supondo que "label" seja a chave que contém as labels

# Aplicando a função de filtro ao dataset
filtered_tomato = ds_train.filter(filtrar_tomates)
list(filtered_tomato.take(1).as_numpy_iterator())

[{'image': array([[[171, 161, 169],
          [158, 148, 156],
          [153, 143, 151],
          ...,
          [144, 133, 137],
          [130, 119, 123],
          [151, 140, 144]],
  
         [[164, 154, 162],
          [158, 148, 156],
          [161, 151, 159],
          ...,
          [131, 120, 124],
          [ 86,  75,  79],
          [104,  93,  97]],
  
         [[150, 140, 148],
          [155, 145, 153],
          [161, 151, 159],
          ...,
          [101,  90,  94],
          [ 90,  79,  83],
          [124, 113, 117]],
  
         ...,
  
         [[176, 165, 169],
          [160, 149, 153],
          [163, 152, 156],
          ...,
          [158, 146, 146],
          [137, 125, 125],
          [112, 100, 100]],
  
         [[171, 160, 164],
          [190, 179, 183],
          [151, 140, 144],
          ...,
          [149, 137, 137],
          [140, 128, 128],
          [105,  93,  93]],
  
         [[142, 131, 135],
          [182, 171, 175],
          [173,

In [12]:
# Fazendo normalização e processamento para as de tomate
ds_tomate = filtered_tomato.map(processamento)

# Fazendo divisão da base de dados do tomate
ds_tomate = ds_tomate.shuffle(1000).prefetch(1)

# Calcular o tamanho do conjunto de validação como 25% dos dados de treinamento
train_size = ds_tomate.cardinality().numpy()
validation_size = train_size // 4  # 25% dos dados de treinamento

# Dividindo os dados
ds_tomate_validation = ds_tomate.take(validation_size)
ds_tomate_train = ds_tomate.skip(validation_size)

# for image, label in ds_tomate_validation.take(1):
#     print("Imagem de treinamento após o processamento: \n{} \ne label {}".format(image.shape, label))

ds_tomate_validation_x = ds_tomate_validation.map(lambda img, lbl: img)
ds_tomate_validation_y = ds_tomate_validation.map(lambda img, lbl: lbl)

ds_tomate_train_x = ds_tomate_train.map(lambda img, lbl: img)
ds_tomate_train_y = ds_tomate_train.map(lambda img, lbl: lbl)


In [10]:
# Diminuindo tamanho do dataset para melhor processamento

# Embaralhando os dados para garantir que tanto a validação quanto o treinamento tenham todos os tipos de dados
ds_train = ds_train.shuffle(buffer_size=1000, seed=42)
filtered_dataset = ds_train.take(20000)
list(filtered_dataset.take(1).as_numpy_iterator())
print(type(filtered_dataset))

<class 'tensorflow.python.data.ops.take_op._TakeDataset'>


In [11]:
# Fazendo normalização e processamento para os dados reduzidos

filtered_dataset = filtered_dataset.map(processamento)

# Fazendo divisão da base de dados do tomate
filtered_dataset = filtered_dataset.shuffle(1000).prefetch(1)

# Calcular o tamanho do conjunto de validação como 25% dos dados de treinamento
train_size = filtered_dataset.cardinality().numpy()
validation_size = train_size // 4  # 25% dos dados de treinamento

# Dividindo os dados
filtered_dataset_validation = filtered_dataset.take(validation_size)
filtered_dataset_train = filtered_dataset.skip(validation_size)

# for image, label in filtered_dataset_validation.take(1):
#     print("Shape da imagem de treinamento após o processamento:{} e label {}".format(image, label))

filtered_dataset_validation_x = filtered_dataset_validation.map(lambda img, lbl: img)
filtered_dataset_validation_y = filtered_dataset_validation.map(lambda img, lbl: lbl)

filtered_dataset_train_x = filtered_dataset_train.map(lambda img, lbl: img)
filtered_dataset_train_y = filtered_dataset_train.map(lambda img, lbl: lbl)


In [25]:
for label in filtered_dataset_train_y.take(5):
  print(label)


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'element_spec'

In [21]:
# Dividir os dados de treinamento em treinamento e validação sem filtragem

ds_train2 = ds_train.map(processamento)

# Embaralhando os dados para garantir que tanto a validação quanto o treinamento tenham todos os tipos de dados
ds_train2 = ds_train2.shuffle(buffer_size=1000, seed=42).prefetch(1)

# Calcular o tamanho do conjunto de validação como 25% dos dados de treinamento
train_size = ds_train2.cardinality().numpy()
validation_size = train_size // 4  # 25% dos dados de treinamento

# Dividindo os dados
ds_validation = ds_train2.take(validation_size)
ds_train2 = ds_train2.skip(validation_size)

# Verificar o número de exemplos em cada conjunto de dados
num_train_examples = ds_train2.cardinality().numpy()
num_validation_examples = ds_validation.cardinality().numpy()
print("Número de exemplos de treinamento:", num_train_examples)
print("Número de exemplos de validação:", num_validation_examples)


Número de exemplos de treinamento: 40728
Número de exemplos de validação: 13575


In [22]:
ds_validation_x = ds_validation.map(lambda img, lbl: img)
ds_validation_y = ds_validation.map(lambda img, lbl: lbl)

ds_train_x = ds_train2.map(lambda img, lbl: img)
ds_train_y = ds_train2.map(lambda img, lbl: lbl)
